# 解决一些来源无法正确获取新闻内容的问题

In [1]:
%%time

from gne import GeneralNewsExtractor
import requests

def get_url_content(url:str) -> str:
    """根据 url 访问并返回内容"""
    response = requests.get(url)
    html=response.text

    return html

CPU times: user 196 ms, sys: 101 ms, total: 297 ms
Wall time: 125 ms


In [3]:
%%time

import requests
import urllib3
import ssl


class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    # "Transport adapter" that allows us to use custom ssl_context.

    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)


def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 32.4 µs


In [15]:
%%time

url="https://politics.gmw.cn/2024-06/12/content_37375817.htm"
response=get_legacy_session().get(url)

response.encoding="utf-8"
response.text[:1000]

CPU times: user 53.6 ms, sys: 0 ns, total: 53.6 ms
Wall time: 77 ms


'<!DOCTYPE html>\r\n<html>\r\n<head> \r\n<meta name="tplInfo" content=\'{"id":46074,"name":"2018文章页全媒体纯净版","nodeid":0}\'>\r\n<script src="//img.gmw.cn/js/public_share.js" atremote></script>\r\n<meta charset="UTF-8" />\r\n<meta http-equiv="Cache-Control" content="max-age=0" />\r\n<meta http-equiv="pragma" content="no-cache" />\r\n<meta http-equiv="Expires" content="0" />\r\n<meta name="renderer" content="webkit|ie-stand|ie-comp">\r\n<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=3.0, user-scalable=no" >\r\n<meta http-equiv="X-UA-Compatible" content="IE=10,chrome=1"/>\r\n<meta name="description" content="近日，中共中央总书记、国家主席、中央军委主席习近平给中国科学院院士、清华大学教授姚期智回信，向他致以诚挚问候并提出殷切希望。">\r\n<meta name="keywords" content="习近平 姚期智 清华大学 科学院院士 教育强国">\r\n<meta name="filetype" content="0">\r\n<meta name="publishedtype" content="1">\r\n<meta name="pagetype" content="1">\r\n<META name="catalogs" content="23717">\r\n<META name="contentid" content="37375817">\r\

In [16]:
%%time

from gne import GeneralNewsExtractor

html=response.text
extractor = GeneralNewsExtractor()
result = extractor.extract(html, noise_node_list=['//div[@class="comment-list"]'])

result

CPU times: user 13.3 ms, sys: 3.99 ms, total: 17.3 ms
Wall time: 16.3 ms


{'title': '习近平给中国科学院院士、清华大学教授姚期智回信',
 'author': '杨煜',
 'publish_time': '2024-06-12',
 'content': '新华社北京6月12日电\n近日，中共中央总书记、国家主席、中央军委主席习近平给中国科学院院士、清华大学教授姚期智回信，向他致以诚挚问候并提出殷切希望。',
 'images': [],
 'meta': {'tplInfo': '{"id":46074,"name":"2018文章页全媒体纯净版","nodeid":0}',
  'renderer': 'webkit|ie-stand|ie-comp',
  'viewport': 'width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=3.0, user-scalable=no',
  'description': '近日，中共中央总书记、国家主席、中央军委主席习近平给中国科学院院士、清华大学教授姚期智回信，向他致以诚挚问候并提出殷切希望。',
  'keywords': '习近平 姚期智 清华大学 科学院院士 教育强国',
  'filetype': '0',
  'publishedtype': '1',
  'pagetype': '1',
  'catalogs': '23717',
  'contentid': '37375817',
  'publishdate': '2024-06-12',
  'author': '杨煜',
  'editor': '杨煜'}}

In [11]:
%%time

url="http://news.china.com.cn/2024-06/13/content_117250849.shtml"
response=get_legacy_session().get(url)

response.text[:1000]

CPU times: user 4.5 ms, sys: 54 µs, total: 4.55 ms
Wall time: 96.5 ms


'\ufeff<!DOCTYPE html>\r\n<html lang="en">\r\n\r\n<head>\r\n    <meta charset="UTF-8" />\r\n    \r\n    \r\n    <meta property="og:type" content="website" />\r\n    <meta property="og:title" content="莫斯科证券交易所宣布停止使用美元和欧元交易">\r\n    <meta property="og:description" content="中国网--网上中国">\r\n    <meta property="og:image" content="http://images.china.cn/images1/ch/2022ChinaIdex/img/shareF2023.png">\r\n    <meta property="og:url" content="http://www.china.com.cn/">\r\n\r\n    \r\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0" />\r\n    <meta http-equiv="X-UA-Compatible" content="ie=edge" />\r\n    <link rel="stylesheet" href="http://images.china.cn/images1/resource/styles/reset-1.0.css" />\r\n    <link rel="stylesheet" href="http://images.china.cn/images1/ch/2022ChinaIdex/nav/css/topMenu.min.css">\r\n    <link rel="stylesheet" href="http://images.china.cn/images1/ch/2022ChinaIdex/content/css/content_two.min.css">\r\n    <link rel

In [12]:
html=response.text
extractor = GeneralNewsExtractor()
result = extractor.extract(html, noise_node_list=['//div[@class="comment-list"]'])

result

{'title': '莫斯科证券交易所宣布停止使用美元和欧元交易',
 'author': '刘洪庆',
 'publish_time': ' 2024-06-13',
 'content': '新华社圣彼得堡6月12日电（记者 安晓萌）俄罗斯莫斯科证券交易所12日发布公告，宣布次日起不再使用美元和欧元进行交易。\n公告说，由于美国对莫斯科证券交易所采取限制措施，莫斯科证券交易所自6月13日起将不再使用美元和欧元进行外汇和贵金属交易。此外，在股票和货币市场以及标准化金融衍生工具市场，也不再使用以美元和欧元结算的金融工具进行交易。\n美国财政部12日宣布对俄罗斯实行新一揽子制裁措施，目标包括俄金融系统的组成机构，以及为俄黄金生产商提供原料以支持俄武器制造的“跨国网络”等。其中，莫斯科证券交易所及其下设的国家结算存管公司以及国家清算中心等在制裁对象名单中。（参与记者：邓仙来）\n【责任编辑：刘洪庆】',
 'images': [],
 'meta': {'og:type': 'website',
  'og:title': '莫斯科证券交易所宣布停止使用美元和欧元交易',
  'og:description': '中国网--网上中国',
  'og:image': 'http://images.china.cn/images1/ch/2022ChinaIdex/img/shareF2023.png',
  'og:url': 'http://www.china.com.cn/',
  'viewport': 'width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0',
  'keywords': '莫斯科,金融,交易,制裁,不再',
  'filetype': '0',
  'publishedtype': '1',
  'pagetype': '1',
  'catalogs': '',
  'contentid': '117250849',
  'publishdate': ' 2024-06-13',
  'author': '刘洪庆',
  'titlePic': ''}}